### Transfer Learning for Custom Datasets in the Small-Data Regime for Basement

# Milestone 1: Environment Preparation
First step is to install a data anotation tool CVAT locally in our laptop.The document below cointains instruction for insalling on MAC OS with following system specifictions :
|  |   |
|--|--|
| Processer | 6-Core Intel i7  |
|Processer Speed|2.6 Ghz|
|Memory|16GB|
|System Version|macOS 12.4|
|Kernal Version|Darwin 21.5.0|


## Quick installation guide
### Install and run Docker Desktop on Mac

 1. Downloaded `Docker.dmg` from this [link](https://desktop.docker.com/mac/main/amd64/Docker.dmg?utm_source=docker&utm_medium=webreferral&utm_campaign=docs-driven-download-mac-amd64)
 2. Double-click  `Docker.dmg`  to open the installer, then drag the Docker icon to the Applications folder.
 3. Double-click  `Docker.app`  in the  **Applications**  folder to start Docker.
 4. The Docker menu (![whale menu](https://docs.docker.com/desktop/install/images/whale-x.svg)) displays the Docker Subscription Service Agreement window. Select **Accept** to continue. 
 5. Docker Desktop starts after you accept the terms.

### Install Git
For me I already had `git` installed. If you don’t have it installed already you can find instructions  [here](https://git-scm.com/book/en/v2/Getting-Started-Installing-Git).

### Download and install Google Chrome
Google Chrome is the only browser which is supported by CVAT.

 1. Go to [Download Chrome](https://www.google.com/chrome/)
 2. Click Download Chrome.
 3. Double-click  `GoogleChrome.dmg`  to open the installer, then drag the Google Chrome icon to the Applications folder.
 4. Double-click  `GoogleChrome.app`  in the  **Applications**  folder to start Google Chrome.

### Installing CVAT

 1. Open a terminl Window.
 2. Clone  _CVAT_  source code from the  [GitHub repository](https://github.com/opencv/cvat)  with Git. The following command will clone the latest develop branch:
  ```shell
git clone https://github.com/opencv/cvat
```
3. Move into _CVAT_ folder .
```shell
cd cvat
```
4. Run docker containers. It will take some time to download the latest CVAT release and other required images like postgres, redis, etc. from DockerHub and create containers.
 ```shell
    docker-compose up -d
   ```
5. You can register a user but by default it will not have rights even to view list of tasks. Thus you should create a superuser. A superuser can use an admin panel to assign correct groups to other users. Please use the command below:
```shell
docker exec -it cvat_server bash -ic 'python3 ~/manage.py createsuperuser'
```
6. Choose a username and a password for your admin account.
7. Open the installed Google Chrome browser and go to [localhost:8080](http://localhost:8080/). Type your login/password for the superuser on the login page and press the _Login_ button. Now you should be able to create a new annotation task.
#### Login Page
<p align="center">
  <img src="https://i.ibb.co/sC4QpPf/bfcb2be8-f9d6-4cd8-bad5-df1a83dbd076.jpg" />
</p>

#### Home Page
<p align="center">
  <img src="https://i.ibb.co/VBTNJNH/ac3e0527-e58f-4928-bec7-69b0da1ab52b.jpg" />
</p>

### Transfer Learning for Custom Datasets in the Small-Data Regime for Basement

# Milestone 2: Data Acquisition

For the basement I shortlisted following 10 product catogries that HD sells and are visible in their video :

 1. Pillar Wood/Steel 
 2. Television 
 3. Photo Frame
 4. Lamp and Lighting 
 5. Fireplace 
 6. Flooring 
 7. Sofa 
 8. Table 
 9. Installation Service 
 10. Window 

I used Google Custom Search API to get images for disered labels. Here is the code for a particular lable :
```python
# !pip install google-api-python-client

import  requests
from  PIL  import  Image
import  os

api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
from apiclient.discovery import build
resource = build("customsearch", 'v1', developerKey=api_key).cse()
items = []

search_string = "basement wire"
cx = "xxxxxxxxxxx"  #Custom Search Engine ID

for  rng  in  range(1,100,10):
	print(rng, end=' ')
	result = resource.list(q=search_string, cx='c027b0cbdfed84f0b', searchType = 'image',start=rng).execute()
	for  item  in  result['items']:
		items.append([item['title'], item['link']])

# Saving Images into folder
dir_name = './wire_data/'
if  dir_name  not  in  os.listdir():
	os.makedirs('dir_name')
	
i=0
for  item  in  items:
	img_url = item[1]
	response = requests.get(img_url)
	if  response.status_code:
		fp = open('./wire/+'+str(i)+'.png', 'wb')
		fp.write(response.content)
		fp.close()
	print(i,item[1])
	i+=1
```



### Transfer Learning for Custom Datasets in the Small-Data Regime for Basement

# Milestone 3: Annotation

## **Deep Extreme Cut: From Extreme Points to Object Segmentation**

DEXTR (Deep Extreme Cut) is a deep learning architecture that is designed to perform object segmentation in images. The architecture is based on the idea of "extreme points," which are the points in an image that are farthest away from the background. By identifying these extreme points and using them as the starting points for object segmentation, DEXTR is able to more accurately segment objects in images.

**![](https://lh6.googleusercontent.com/tcig1ZD6GH0imaEwYKfZbeBaTl0O1kdnBaEHtYsc8Qzn5CMGAi_poKhNtQDxk8R4Fu8Sg3x8ABQ28Ao5avU-IEMl7hn2ICrCvlNOhIMmS6ZyNZmRBfKIkeY0tRM87xPXsTrX-71Yr-yjQwvydCj-XK4yoRU-OEeAsS3mU7cCPTs2UmXtRAeBwMVVRZ3eHQ)**

The architecture of DEXTR consists of two main components: a feature extraction network and a decoder network. The feature extraction network is typically a pre-trained deep convolutional neural network (CNN), such as VGG-16 or ResNet-101, that is used to extract high-level features from the input image. The decoder network, on the other hand, is a fully-connected network that takes the output of the feature extraction network and generates a segmentation map for the input image.

  

## Feature Extraction:

  

A 4-channel input for the CNN is created in the DEXTR architecture by concatenating the heatmap with the RGB channels of the input image. A bounding box created from the extreme point annotations is used to crop this input. The bounding box is then relaxed by a few pixels to include context surrounding the object of interest. As a result of this preprocessing step, the input is an RGB crop that includes the object and its extreme points.

  

The ResNet-101 is used to extract the features. The fully connected layer and the last 2 max pooling layer are removed to get the required resolution(atrous convolution). After the ResNet, a pyramid scene based parsing module is designed to collect globally valuable features. The weights of the network are pre-defined using ImageNet to give better results. The output of the feature extraction part is the probability map. It represents the probability of each pixel belonging to an object.

  

  

CNN tries to minimize the given cost function. The given cost function is called the entropy loss function between the label and the prediction $\hat{y}_j$

  
  

$$\mathcal{L}=\sum_{j \in Y} w_{Y_j} C\left(y_j, \hat{y}_j\right), \quad j \in  1, \ldots,|Y|$$

  
  

## Decoder Network:

  

To generate the segmentation map, the decoder network first identifies the extreme points in the input image using a set of predefined rules. These points are then used as the starting points for a graph-based segmentation algorithm, which is used to generate the final segmentation map. This map is then refined using a set of post-processing steps, such as removing small objects and filling in gaps in the segmentation, to produce the final segmentation map for the input image.

  
  

## Interactive Segmentation:

  

Interactive segmentation is a method of segmenting objects in images that involves providing user input to guide the segmentation process. In the context of the Deep Extreme Cut architecture, interactive segmentation may be performed by providing the model with "extreme points" as input, which are the points in an image that are farthest away from the background. These points are used as the starting points for the DEXTR model to segment the objects in the image, allowing the user to guide the segmentation process by selecting the points that are most relevant to the task at hand.

  

  

The interactive segmentation can improve the accuracy and efficiency of object segmentation by allowing the user to provide guidance to the model, and can be a useful tool in situations where the objects of interest are not well-defined or are difficult to segment automatically.

  

## Conclusion

  

To conclude, the DEXTR architecture combines the strengths of both convolutional neural networks and graph-based methods to achieve state-of-the-art performance on a variety of object segmentation tasks. Due to this, there has been widespread adoption in the research community.


# Annotation Link
The dataset is uploaded on drive at this [Basement Data Set](https://drive.google.com/drive/folders/1bQI5NGNnWryRgPsNFFRrRT2tpAoFJZl2?usp=sharing)


### Transfer Learning for Custom Datasets in the Small-Data Regime for Basement

# Milestone 4: Semantic Segmentation
![Sample Segementation](https://i.ibb.co/sH6FX64/6a745d3b-952d-49de-9946-fde2a6b024f1.jpg)

The architecture was inspired by  [U-Net: CNN for Biomedical Image Segmentation](http://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/).
## Network Description

### Data

The pretrained data is from ADK20k images dataset fine tuned for our scraped images to be found here [Basement Data Set](https://drive.google.com/drive/folders/1bQI5NGNnWryRgPsNFFRrRT2tpAoFJZl2?usp=sharing)

### Model

[![img/u-net-architecture.png](https://github.com/zhixuhao/unet/raw/master/img/u-net-architecture.png)](https://github.com/zhixuhao/unet/blob/master/img/u-net-architecture.png)

This deep neural network is implemented with Keras functional API, which makes it extremely easy to experiment with different interesting architectures.

Output from the network is a 512*512 which represents mask that should be learned. Sigmoid activation function makes sure that mask pixels are in [0, 1] range.

### Training

The model is trained for 5 epochs.

After 5 epochs, calculated accuracy is about 0.97.

Loss function for the training is basically just a binary crossentropy.

## How to Use
### Dependencies

This tutorial depends on the following libraries:
-   Hardware: >=4 GPUs for training, >=1 GPU for testing (set  `[--gpus GPUS]`  accordingly)
-   Software: Ubuntu 16.04.3 LTS,  _**CUDA>=8.0, Python>=3.5, PyTorch>=0.4.0**_
-   Dependencies: numpy, scipy, opencv, yacs, tqdm

Also, this code should be compatible with Python versions 2.7-3.5.

1.  Here is a simple demo to do inference on a single image:
```
chmod +x demo_test.sh
./demo_test.sh
```
This script downloads a trained model (UNet50dilated) and a test image, runs the test script, and saves predicted segmentation (.png) to the working directory.

2.  To test on an image or a folder of images (`$PATH_IMG`), you can simply do the following:

```python
python3 -u test.py --imgs $PATH_IMG --gpu $GPU --cfg $CFG

```

## Results 

### Preformance 

| Architecture | Mean IoU |Pixel Accuracy(%) |Overall Score |Inference Speed(fps) |
|--|--|--|--|--|
| UNet50dilated | 42.14 |80.13 |61.14 | 2.6|
|UNet18dilated|38.00|78.64	|58.32|11.7|

### Image Output
![enter image description here](https://i.ibb.co/t2ckyq1/1813462c-15a7-44a3-a0ed-421e9482ab64.jpg)

![enter image description here](https://i.ibb.co/HdQpYH7/0e416353-5d97-454f-affa-9c8deb5563a2.jpg)

![enter image description here](https://i.ibb.co/VNhHDGk/04a054c1-74b9-4339-9f59-edcbb3017ee6.jpg)